# Links

<a href="https://huggingface.co/Helsinki-NLP/opus-mt-en-ar">Model Link</a>

<a href="https://huggingface.co/datasets/ymoslem/CoVoST2-EN-AR-Text">Data Link</a>

### Install Transformars

In [1]:
!pip install datasets transformers[sentencepiece] sacrebleu -q

# Install DataSet

In [2]:
from datasets import load_dataset,Dataset

ds = load_dataset("ymoslem/CoVoST2-EN-AR-Text")

README.md:   0%|          | 0.00/952 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/28.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/269380 [00:00<?, ? examples/s]

# Install Model And Tokenizer

In [3]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-ar")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-ar")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/801k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/917k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.12M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

# Know more about the data

In [4]:
ds

DatasetDict({
    train: Dataset({
        features: ['text_en', 'text_ar'],
        num_rows: 269380
    })
})

In [5]:
ds['train']

Dataset({
    features: ['text_en', 'text_ar'],
    num_rows: 269380
})

In [6]:
ds['train'][10:20]

{'text_en': ['That still boggles my mind.\n',
  'The Embarrass River runs east-west through the middle of Waasa Township.\n',
  'Aguer is considered a representative of the conservative sector of the Argentine Church.\n',
  'She also smashes plates everywhere with crockery and kitchen utensils.\n',
  'The community most likely has the name of James Gunn, a pioneer citizen.\n',
  'They run calculations and realize that the matter may actually be a tiny universe.\n',
  'She should have stuck with it.\n',
  'It is believed they were printed by rubbing pressure, rather than a printing press.\n',
  'As the guards go to find the farmhands, Kira attempts to reason with Mullibok.\n',
  'Nothing is easier than getting a bad habit!\n'],
 'text_ar': ['هذا لا يزال يحير ذهني.\n',
  'يمتد نهر إمباراس من الشرق إلى الغرب عبر وسط بلدة واسا.\n',
  'يُعتبر «أوغر» ممثلًا للمحافظين في الكنيسة الأرجنتينية.\n',
  'تحطم كذلك الأطباق في كل مكان مع الأواني الفخارية وأدوات المطبخ.\n',
  'من المرجح أن يطلق على ال

In [7]:
ds_copy = ds.copy()

# Shuffle the data

#### To ensure the data not ordered by topic or complexity

In [8]:
New_ds = ds["train"].shuffle(seed=42)

# Splitting data

#### Split the data to train-validate-test

In [9]:
split_ds = New_ds.train_test_split(test_size=0.2, seed=42)

train_ds = split_ds["train"]

temp_ds = split_ds["test"]



val_test_split = temp_ds.train_test_split(test_size=0.5, seed=42)

val_ds = val_test_split["train"]

test_ds = val_test_split["test"]



print(f"Train size: {len(train_ds)}, Validation size: {len(val_ds)}, Test size: {len(test_ds)}")

Train size: 215504, Validation size: 26938, Test size: 26938


In [10]:
split_ds

DatasetDict({
    train: Dataset({
        features: ['text_en', 'text_ar'],
        num_rows: 215504
    })
    test: Dataset({
        features: ['text_en', 'text_ar'],
        num_rows: 53876
    })
})

In [11]:
train_ds[0]

{'text_en': 'The pilot was Brian Binnie.\n',
 'text_ar': 'كان الطيار بريان بيني.\n'}

# Data cleaning

* check nulls

* check duplicates

* Lower_case

* switch contractinos

* remove unwanted punctuations

In [12]:
import pandas as pd

df=pd.DataFrame(ds['train'])

df.isna().sum()

text_en    0
text_ar    0
dtype: int64

In [13]:
df.duplicated().sum()

0

# install contractions library

In [14]:
!pip install contractions

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 9.0 MB/s eta 0:00:00


In [15]:
import re

import contractions



def to_lowercase(text,lower=True):

    return text.lower()



def remove_newlines(text):

     return text.replace('\n', '')



def fix_contractions(text):

    return contractions.fix(text)



def remove_unwanted_punctuation(text):

    return re.sub(r'[^\w\s.,!?\'\"؛،]', '', text)



def preprocess_text(text,lower=True):

    text = to_lowercase(text,lower)

    text = remove_newlines(text)

    text = fix_contractions(text)

    text = remove_unwanted_punctuation(text)

    return text



def preprocess_dataset(examples,lower=True):

    examples['text_en'] = [preprocess_text(text, lower) for text in examples['text_en']]

    examples['text_ar'] = [preprocess_text(text, lower) for text in examples['text_ar']]

    return examples



def dataset(dataset, lower=True):

    return dataset.map(preprocess_dataset, batched=True, fn_kwargs={'lower': lower})

In [16]:
train_ds=dataset(train_ds)

val_ds=dataset(val_ds)

test_ds=dataset(test_ds)

Map:   0%|          | 0/215504 [00:00<?, ? examples/s]

Map:   0%|          | 0/26938 [00:00<?, ? examples/s]

Map:   0%|          | 0/26938 [00:00<?, ? examples/s]

In [17]:
train_ds[:8]

{'text_en': ['the pilot was brian binnie.',
  'there is a craft industry where wooden souvenirs can be purchased in shops.',
  'when it started to get serious, some of them just threw up their hands.',
  'the man in the blue jacket is walking in a crowd of people past an english pub.',
  'selenium webdriver provides a powerful but simple api to control webpages.',
  'the main economical activity in spaoh is farming.',
  'property is being redeveloped for civilian use as brunswick landing.',
  'adjustable buttons allow tuning.'],
 'text_ar': ['كان الطيار بريان بيني.',
  'هناك صناعات حرفية حيث يمكن شراء التذكارات الخشبية من المتاجر.',
  'عندما بدأ الأمر يصبح جديا، تملص البعض منهم مباشرة.',
  'يسير الرجل ذو السترة الزرقاء وسط حشد من الناس أمام حانة إنجليزية.',
  'يوفر ويب درايفر سيلينيوم واجهة برمجة تطبيقات قوية ولكنها بسيطة للتحكم بصفحات الإنترنت.',
  'النشاط الاقتصادي الرئيسي في سبواه هو الزراعة.',
  'تتم إعادة تطوير الممتلكات للاستخدام المدني باسم brunswick landing.',
  'يمكنك الضبط من

# Preprocessing

In [18]:
tokenizer("hello marwan")

{'input_ids': [19043, 27594, 15011, 0], 'attention_mask': [1, 1, 1, 1]}

In [19]:
def preprocess_function(examples):

    inputs = [ex for ex in examples['text_en']]

    targets = [ex for ex in examples['text_ar']]

    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")



    with tokenizer.as_target_tokenizer():

        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")



    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [20]:
train_ds = train_ds.map(preprocess_function, batched=True)

val_ds = val_ds.map(preprocess_function, batched=True)

test_ds = test_ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/215504 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/26938 [00:00<?, ? examples/s]

Map:   0%|          | 0/26938 [00:00<?, ? examples/s]

### Make Batches and handling padding lengths

In [21]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [22]:
data_collator

DataCollatorForSeq2Seq(tokenizer=MarianTokenizer(name_or_path='Helsinki-NLP/opus-mt-en-ar', vocab_size=62802, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	62801: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}, model=MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(62802, 512, padding_idx=62801)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(62802, 512, padding_idx=62801)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (sel

# Fine-Tuning

In [23]:
subset_train_ds = train_ds.shuffle(seed=42).select([i for i in range(int(len(train_ds) * 0.3480213824))])
subset_val_ds = val_ds.shuffle(seed=42).select([i for i in range(int(len(val_ds) * 0.185611404))])

In [24]:
subset_test_ds = test_ds.shuffle(seed=42).select([i for i in range(int(len(test_ds) * 0.001))])

In [25]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import EarlyStoppingCallback
from transformers import Seq2SeqTrainingArguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=3,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True
)
trainer = Seq2SeqTrainer(
    model=model,                         # Pre-trained model
    args=training_args,                  # Training arguments
    train_dataset=train_ds,              # Training dataset
    eval_dataset=val_ds,                 # Validation dataset
    tokenizer=tokenizer,                 # Corrected from 'preprocessing_class' to 'tokenizer'
    data_collator=data_collator,         # Data collator for padding
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # Early stopping
)

/tmp/ipykernel_23/4260081858.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [26]:
subset_train_ds.shape

(74999, 5)

In [27]:
subset_val_ds.shape

(5000, 5)

In [28]:
subset_test_ds.shape

(26, 5)

In [29]:
import wandb
wandb.login(key="b2486d3312fd17fb4c516de3c7dfd2e9b8f285a2")


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [30]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: marwanmohamed200415 (marwanmohamed200415-cairo-university). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.18.7
wandb: Run data is saved locally in /kaggle/working/wandb/run-20241207_180759-0bx2r7x3
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run ./results
wandb: ⭐️ View project at https://wandb.ai/marwanmohamed200415-cairo-university/huggingface
wandb: 🚀 View run at https://wandb.ai/marwanmohamed200415-cairo-university/huggingface/runs/0bx2r7x3


Epoch,Training Loss,Validation Loss
1,0.222600,0.185460
2,0.194400,0.175709
3,0.121900,0.171204


/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62801]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.encoder.embed_positions.weight', 'model.decoder.embed_tokens.weight', 'model.decoder.embed_positions.weight', 'lm_head.weight'].


TrainOutput(global_step=161628, training_loss=0.17189722941670665, metrics={'train_runtime': 12241.139, 'train_samples_per_second': 52.815, 'train_steps_per_second': 13.204, 'total_flos': 2.1915699706331136e+16, 'train_loss': 0.17189722941670665, 'epoch': 3.0})

In [31]:
import torch
def translate_text(input_text, model, tokenizer, device=None):
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True).to(device)
    outputs = model.generate(**inputs)
    translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translated_text

In [32]:
translate_text('Both cases are clearly visible in Iran.', model, tokenizer, device=None)

'يمكن رؤية الحالات الفرعية بوضوح في إيران.'

In [33]:
trainer.save_model("model_directory")
tokenizer.save_pretrained("model_directory")

('model_directory/tokenizer_config.json',
 'model_directory/special_tokens_map.json',
 'model_directory/vocab.json',
 'model_directory/source.spm',
 'model_directory/target.spm',
 'model_directory/added_tokens.json')

In [34]:
from nltk.translate.bleu_score import corpus_bleu

# Prepare lists to collect references and predictions
references = []
predictions = []

for i in range(520, 570):
    print(f"The English sentence: {test_ds['text_en'][i]}")
    print(f"Its Arabic translation: {test_ds['text_ar'][i]}")
    
    # Model translation
    translated = translate_text(test_ds['text_en'][i], model, tokenizer, device=None)
    print(f"The Model translation: {translated}")
    print("-----------------------------------------------")
    print('\n')
    
    # Collect the reference (ground truth) and the prediction
    references.append([test_ds['text_ar'][i].split()])  # Reference should be a list of words
    predictions.append(translated.split())  # Prediction should be a list of words

# After processing all samples, compute the BLEU score
bleu_score = corpus_bleu(references, predictions)
print(f"BLEU score: {bleu_score}")


The English sentence: windows live home became the official home portal for all windows live services instead.
Its Arabic translation: أصبح windows live home البوابة الرئيسية الرسمية لجميع خدمات windows live بدلا من ذلك.
The Model translation: أصبحت windows live home البوابة الرئيسية لجميع خدمات windows live بدلا من ذلك.
-----------------------------------------------


The English sentence: "i do not want to."
Its Arabic translation: لا أريد فعل ذلك.
The Model translation: لا أريد ذلك.
-----------------------------------------------


The English sentence: also seen in neopaganism and ceremonial magic.
Its Arabic translation: وينظر أيضا في neopaganism والسحر الاحتفالي.
The Model translation: يمكن رؤيته أيضا في الهجاء الجديد والسحر الاحتفالي.
-----------------------------------------------


The English sentence: blinman also recorded and mixed the album, with brahl assisting with additional engineering.
Its Arabic translation: قام بلينمان أيضا بتسجيل الألبوم ومزجه، مع براهل المساعدة ف

In [35]:
model_save_path = "./my_model"
tokenizer_save_path = "./my_tokenizer"
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(tokenizer_save_path)
print(f"Model and tokenizer saved at {model_save_path} and {tokenizer_save_path}")

Model and tokenizer saved at ./my_model and ./my_tokenizer


In [36]:
import shutil
shutil.make_archive('/kaggle/working/my_model', 'zip', './my_model')
shutil.make_archive('/kaggle/working/my_tokenizer', 'zip', './my_tokenizer')

'/kaggle/working/my_tokenizer.zip'

In [37]:
translate_text('i love egypt', model, tokenizer, device=None)

'.أحب مصر'